In [ ]:
# reference for footnote is <a class="noteref" epub:type="noteref" href="#note2">2</a>

# footnotes in ibooks is 
# <aside class="footnote" epub:type="footnote" id="note1">1:<img alt="" class="alignnone1" loading="lazy" src="https://thepotatoroom.com/wp-content/uploads/2019/11/1280px-Archaeopteryx_Philadelphia-300x213.jpg"/></aside>



In [ ]:
# ToC

'''
original:

<navPoint id="num_28" playOrder="28">
      <navLabel>
        <text>Chapter 15.1 The Reader and Protagonist Definitely Have to Be in True Love</text>
      </navLabel>
      <content src="book2_split_027.html"/>
      <navPoint id="num_29" playOrder="29">
        <navLabel>
          <text>Chapter 15.2 The Reader and Protagonist Definitely Have to Be in True Love</text>
        </navLabel>
        <content src="book2_split_028.html"/>
      </navPoint>
    </navPoint>
    


should be:

<navPoint id="num_1" playOrder="1">
      <navLabel>
        <text>Chapter 1</text>
      </navLabel>
      <content src="book2_split_002.html"/>
      <navPoint id="num_2" playOrder="2">
        <navLabel>
          <text>Chapter 1.1 The Reader and Protagonist Definitely Have to Be in True Love</text>
        </navLabel>
        <content src="book2_split_001.html"/>
      </navPoint>
      <navPoint id="num_3" playOrder="3">
        <navLabel>
          <text>Chapter 1.2 The Reader and Protagonist Definitely Have to Be in True Love</text>
        </navLabel>
        <content src="book2_split_002.html"/>
      </navPoint>
    </navPoint>
    
    
<?xml version='1.0' encoding='utf-8'?>
<ncx xmlns="http://www.daisy.org/z3986/2005/ncx/" version="2005-1" xml:lang="en">
  <head>
    <meta content="2a386fbc-b585-41e1-b7bf-1e1d411a79d8" name="dtb:uid"/>
    <meta content="3" name="dtb:depth"/>
    <meta content="calibre (3.48.0)" name="dtb:generator"/>
    <meta content="0" name="dtb:totalPageCount"/>
    <meta content="0" name="dtb:maxPageNumber"/>
  </head>
  <docTitle>
    <text>Reader and Protagonist</text>
  </docTitle>
  <navMap>
      <navPoint id="num_28" playOrder="28">
          <navLabel>
            <text>Chapter 15.1 The Reader and Protagonist Definitely Have to Be in True Love</text>
          </navLabel>
          <content src="book2_split_027.html"/>
          <navPoint id="num_29" playOrder="29">
            <navLabel>
              <text>Chapter 15.2 The Reader and Protagonist Definitely Have to Be in True Love</text>
            </navLabel>
            <content src="book2_split_028.html"/>
          </navPoint>
        </navPoint>
  
  </navmap>
</ncx>
    

In [ ]:
# import re

def getHTML(Soup):
    iWant = soup.find(attrs={"id" : "content"})
    iWant = iWant.get_text()
#     print(iWant.encode("utf-8"))
#     print(iWant.encode("utf-8").decode("utf-8"))
    return iWant

def getLink(Soup):
    tags = soup('a')
    for i in tags:
        if i is not None and len(i) > 0:
            if i.contents[0] == "下一章":
                return "https://www.shubaow.net"+i.get('href', None)
    return False

def download(page, file_name, chapter_title):
    file = open(file_name, "w", encoding='gb2312')
    file.write('<html xmlns="http://www.w3.org/1999/xhtml">')
    file.write("\n<head>")
    file.write("\n<title>" + chapter_title + "</title>")
    file.write("\n</head>")
    file.write("\n<body>")
    file.write("\n<strong>" + chapter_title + "</strong>" + "\n<p>")
    file.write(page)
    file.write("</p>")
    file.write("\n</body>")
    file.write("\n</html>")
    file.close()
    
def download(page, file_name):
    print(type(page))
    file = open(file_name, "w")
    print(page)
    file.write('<html xmlns="http://www.w3.org/1999/xhtml">')
    file.write("\n<head>")
    file.write(' <meta http-equiv="Content-Type" content="text/html; charset=UTF-8" /> ')
    file.write("\n</head>")
    file.write("\n<body>")
    file.write("\n<p>")
    file.write(page)
    file.write("</p>")
    file.write("\n</body>")
    file.write("\n</html>")
    file.close()

In [ ]:
from urllib.request import urlopen, Request
# import pypub
# import praw, os
import os

from bs4 import BeautifulSoup
import ssl

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}

reg_url = input('Enter - ')
req = Request(url=reg_url, headers=headers) 
html = urlopen(req, context=ctx).read()
soup = BeautifulSoup(html, "html.parser")

urlList =[]
Count = 0

while reg_url and Count < 5:
    download(getHTML(soup), str(Count) + ".html")
    reg_url = getLink(soup)
    if reg_url:
        
#         urlList.append(reg_url)                      # create url list of next chapters
        req = Request(url=reg_url, headers=headers) 
        html = urlopen(req, context=ctx).read()
        soup = BeautifulSoup(html, "html.parser")
    else:
        break;
    Count += 1
        

In [ ]:
# word press clean 
# https://thepotatoroom.com/novels/demonking/demonking-1/

def getLink(Soup):
    try:
        link = soup.findAll('a',attrs={'rel':'next'})[0]['href']
    except:
        return False
    
    if link != None:
        return link
    return False

def getHTML(Soup): # clean up
    for p in soup.find_all('p'):
        if 'style' in p.attrs:
            p.decompose()
    header = soup.find(itemprop="headline")
    iWant = soup.find(itemprop="text")
    removals = iWant.find_all('div', {'class':'saboxplugin-wrap'})
    for match in removals:
        match.decompose()
    print(iWant)
    try:
        footnotes = iWant.find_all('span', {'class':'modern-footnotes-footnote__note'})
        footnoteStr = '\n'.join([str(elem) for elem in footnotes]) 
        
        print("footnotes are:", footnoteStr)
        if footnotes != None:
            for i in footnotes:
                i.decompose()
            return str(header) + "\n" + str(iWant) +"\n" + footnoteStr
        else:
            return str(header) + "\n" + str(iWant)
    except:
        return str(header) + "\n" + str(iWant)

def getChapterTitle(Soup):
    title = soup.find('title')
    return title.string

def createBook(file_name):
    file = open(file_name, "w", encoding="utf8")
    file.write('<?xml version="1.0" encoding="UTF-8" standalone="no"?>')
    file.write('\n<!DOCTYPE html>')
    file.write('\n<html xmlns:epub="http://www.idpf.org/2007/ops">')
    
    file.write("\n<head>")
    file.write('\n<meta charset="UTF-8"/>')
    file.write('\n<link href="epub_style_sheet.css" rel="stylesheet" type="text/css" />')
    file.close()

def download(page, file_name, chapter_title, Count):
    file = open(file_name, "a+", encoding="utf8")
    file.write("\n<title>" + chapter_title + "</title>")
    file.write("\n</head>")
    file.write("\n<body>")
    file.write("\n<p id = chapter{}>".format(str(Count)))
    file.write("\n<strong>" + chapter_title + "</strong>" + "\n<p>")
    file.write(page)
    file.write("</p>")
    file.write("\n</body>")
    file.write("\n</html>")
    file.close()
    
    
def download(page, file_name, Count):
    file = open(file_name, "a+", encoding="utf8")
    file.write("\n</head>")
    file.write("\n<body>")
    file.write("\n<p id = chapter{}>".format(str(Count)))
    file.write(page)
    file.write("</p>")
    file.write("\n</body>")
    file.write("\n</html>")
    file.close()

In [ ]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import ssl

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}

reg_url = input('Enter - ')
req = Request(url=reg_url, headers=headers) 
html = urlopen(req, context=ctx).read()
soup = BeautifulSoup(html, "html.parser")

urlList =[]
Count = 0

file_name = "book1" + ".html"
createBook(file_name)

while reg_url and Count <5:
    
    iWant = getHTML(soup)
    title = getChapterTitle(soup)
    
    if title !=None:
        download(iWant, file_name, Count)
    else:
        download(iWant, file_name, title, Count)
    
    reg_url = getLink(soup)#, Count)
    if reg_url:
        urlList.append(reg_url)                      # create url list of next chapters
        req = Request(url=reg_url, headers=headers) 
        html = urlopen(req, context=ctx).read()
        soup = BeautifulSoup(html, "html.parser")
    else:
        break;
    Count +=1
    
    # http://bcnovels.com/reader-protagonist-definitely-true-love/reader-protagonist-definitely-true-love-chapter-1-1/
    

In [ ]:
import re

number = 1

def getLink(Soup):
    tags = soup('a')
    for i in tags:
        if i is not None and len(i) > 0:
            if i.contents[0] == "Next Chapter":
                return i.get('href', None)
    return False
    

def getHTML(Soup): # clean up
    global number
    for p in soup.find_all('p'):
        if 'style' in p.attrs:
            p.decompose()
    
    
    iWant = soup.find(itemprop="text")
    header = iWant.find("h3")
    print(header)
    removals = iWant.find_all('span', {'class':'easy-footnote-margin-adjust'})
    for match in removals:
        match.decompose()
        
    removals2 = iWant.find_all('div', {'class':'sharedaddy sd-sharing-enabled'})
    for match2 in removals2:
        match2.decompose()
        
        
    removals3 = iWant.find_all('div', {'class':'sharedaddy sd-block sd-like jetpack-likes-widget-wrapper jetpack-likes-widget-loaded'})
    for match3 in removals3:
        match3.decompose()
        
    footnotesList = iWant.find_all('span', {'class':'easy-footnote-margin-adjust'})
        
    
#     try:
    element = iWant.find_all('span', {'class':'easy-footnote'})
    element2 = iWant.find_all('li')
    
    for i in range(len(element)):
        element[i].replace_with('<a class="noteref" epub:type="noteref" href="#note{}">{}</a>'.format(str(number), str(number)))
        element2[i].replace_with('\n<aside class="footnote" epub:type="footnote" id="note{}">{}:'.format(str(number), str(number)) + element2[i].text + '</aside>')
        number +=1

    return "\n" + str(iWant)
#     except:
#         if header == None:
#             return ("header is none")
#         elif iWant == None:
#             return ("iWant is none")
#         else:
#             return ("Neither?")
#         return str(header) + "\n" + str(iWant)

def getChapterTitle(Soup):
    title = soup.find("strong").get_text()
    return title

    

def createBook(file_name):
    file = open(file_name, "w", encoding="utf8")
    file.write('<?xml version="1.0" encoding="UTF-8" standalone="no"?>')
    file.write('\n<!DOCTYPE html>')
    file.write('\n<html xmlns:epub="http://www.idpf.org/2007/ops">')
    
    file.write("\n<head>")
    file.write('\n<meta charset="UTF-8"/>')
    file.write('\n<link href="epub_style_sheet.css" rel="stylesheet" type="text/css" />')
    file.close()

def download(page, file_name, chapter_title, Count):
#     number = 1
    file = open(file_name, "a+", encoding="utf8")
#     file.write("\n<title>" + chapter_title + "</title>")
#     file.write("\n</head>")
    file.write("\n<body>")
    file.write("\n<p id = {}>".format(chappieList[Count]))
    file.write("\n<strong>" + chapter_title + "</strong>" + "\n<p>")
    file.write(page)
    file.write("</p>")
    file.write("\n</body>")
    file.write("\n</html>")
    file.close()
    
    
def download(page, file_name, Count):
    file = open(file_name, "a+", encoding="utf8")
#     file.write("\n</head>")
    if (float(chappieList[Count][8:11]) - 0.1).is_integer():
        file.write("\n<body id = ChapterHead>")
    else:
        file.write("\n<body>")
    file.write("\n<p id = " + str(chappieList[Count]) + "/>")
    if len(chappieList[Count]) > 10:
        file.write("\n<h2>" + str(chappieList[Count][:11]) + "</h2>")
    else:
        file.write("\n<h2>" + str(chappieList[Count][:9]) + "</h2>")
    file.write(page)
    file.write("</p>")
    file.write("\n</body>")
    file.write("\n</html>")
    file.close()
    
# def makeContents(soup):
#     file = open('ToC.ncx', "a+", encoding="utf8")
#     file.write('''
#                 <?xml version='1.0' encoding='utf-8'?>
#                 \n<ncx xmlns="http://www.daisy.org/z3986/2005/ncx/" version="2005-1" xml:lang="en">
#                   \n<head>
#                     \n<meta content="2a386fbc-b585-41e1-b7bf-1e1d411a79d8" name="dtb:uid"/>
#                     \n<meta content="3" name="dtb:depth"/>
#                     \n<meta content="calibre (3.48.0)" name="dtb:generator"/>
#                     \n<meta content="0" name="dtb:totalPageCount"/>
#                     \n<meta content="0" name="dtb:maxPageNumber"/>
#                   \n</head>
#                   \n<docTitle>
#                     \n<text>Reader and Protagonist</text>
#                   \n</docTitle>
#                   \n<navMap>
#                   ''')
#     file.write("\n</head>")
#     file.close()
    
    
# def makeNavigation(chappieList):
#     file = open('ToC.ncx', "a+", encoding="utf8")
#     file.write('<navPoint id="num_1" playOrder="1">')
    
#     file.write('\n<navLabel><text>Chapter {}</text></navLabel>'.format())
#     file.write('<content src="book2_split_{}.html"/>'.format()) # same as first chapter
#     file.write
#       <navPoint id="num_2" playOrder="2">
#         <navLabel>
#           <text>Chapter 1.1 The Reader and Protagonist Definitely Have to Be in True Love</text>
#         </navLabel>
#         <content src="book2_split_001.html"/>
#       </navPoint>
#       <navPoint id="num_3" playOrder="3">
#         <navLabel>
#           <text>Chapter 1.2 The Reader and Protagonist Definitely Have to Be in True Love</text>
#         </navLabel>
#         <content src="book2_split_002.html"/>
#       </navPoint>
#     </navPoint>
    
# def generate(chapt, count):
#     file.write('<navPoint id="num_{}" playOrder="{}">'.format(count))
#     file.write('<navLabel><text>{}</text></navLabel>'.format(chapt))
#     file.write('<content src="book2_split_{:03d}.html"/></navPoint>'.format(count))

In [ ]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import ssl

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}

reg_url = input('Enter - ')
req = Request(url=reg_url, headers=headers) 
html = urlopen(req, context=ctx).read()
soup = BeautifulSoup(html, "html.parser")
urlList =[]
chappieList =[]
ChaptNameList = []
ChaptNameList = soup.find_all("strong")[3:97]
tags = soup('a')
for i in tags:
    
    if i is not None and len(i) > 0:
        if "Chapter" in i.contents[0]: 
            if float(i.contents[0][8:11])<=96:
                chappieList.append(i.contents[0]) # Chapter 1.1
                urlList.append(i.get('href', None))


file_name = "book2" + ".html"
createBook(file_name)
Count =0


# chapterDict = {}

# for i in range(len(chappieList)):
#     if int(chappieList[i+1][8:]) -int(chappieList[i][8:]) <1
#         chapterDict[ChaptNameList[0]]
#         dict[key].append(value)
#         print dict

for i in urlList: #and Count <5:
#     footnotesList = []
    
    req = Request(url=i, headers=headers) 
    html = urlopen(req, context=ctx).read()
    soup = BeautifulSoup(html, "html.parser")
    
    iWant = getHTML(soup)
#     title = getChapterTitle(soup)
    
#     if title !=None:
    download(iWant, file_name, Count)
#     else:
#         download(iWant, file_name, title, Count)
    
#     reg_url = getLink(soup)#, Count)                # create url list of next chapters
    
    Count +=1
    
# http://bcnovels.com/reader-protagonist-definitely-true-love/

In [ ]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import ssl

number = 0
def getHTML(Soup): # clean up
    global number
    for p in soup.find_all('p'):
        if 'style' in p.attrs:
            p.decompose()
    
    
    iWant = soup.find(itemprop="text")
    header = iWant.find("h3")
    print(header)
    removals = iWant.find_all('span', {'class':'easy-footnote-margin-adjust'})
    for match in removals:
        match.decompose()
        
    removals2 = iWant.find_all('div', {'class':'sharedaddy sd-sharing-enabled'})
    for match2 in removals2:
        match2.decompose()
        
        
    removals3 = iWant.find_all('div', {'class':'sharedaddy sd-block sd-like jetpack-likes-widget-wrapper jetpack-likes-widget-loaded'})
    for match3 in removals3:
        match3.decompose()
        
    footnotesList = iWant.find_all('span', {'class':'easy-footnote-margin-adjust'})
        
    
#     try:
    element = iWant.find_all('span', {'class':'easy-footnote'})
    element2 = iWant.find_all('li')
    
    for i in range(len(element)):
        element[i].replace_with('<a class="noteref" epub:type="noteref" href="#note{}">{}</a>'.format(str(number), str(number)))
        element2[i].replace_with('\n<aside class="footnote" epub:type="footnote" id="note{}">{}:'.format(str(number), str(number)) + element2[i].text + '</aside>')
        number +=1

    return "\n" + str(iWant)

def createBook(file_name):
    file = open(file_name, "w", encoding="utf8")
    file.write('<?xml version="1.0" encoding="UTF-8" standalone="no"?>')
    file.write('\n<!DOCTYPE html>')
    file.write('\n<html xmlns:epub="http://www.idpf.org/2007/ops">')
    
    file.write("\n<head>")
    file.write('\n<meta charset="UTF-8"/>')
    file.write('\n<link href="epub_style_sheet.css" rel="stylesheet" type="text/css" />')
    file.close()

def download(page, file_name):
    file = open(file_name, "a+", encoding="utf8")
#     file.write("\n</head>")
    file.write("\n<body>")
    file.write("\n<p id = " + "Chapter SE" + "/>")
    file.write("\n<h2>" + "Chapter SE." + "</h2>")
    file.write("\n<h2>" + "Special Episode: Four Years pt.2" + "</h2>")
    file.write(page)
    file.write("</p>")
    file.write("\n</body>")
    file.write("\n</html>")
    file.close()
    
    
# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}


file_name = "book2pg6" + ".html"
createBook(file_name)

urlList = ["http://bcnovels.com/chapter-33-5-part-2-reader-protagonist-definitely-true-love/"]

for i in urlList: #and Count <5:
#     footnotesList = []
    
    req = Request(url=i, headers=headers) 
    html = urlopen(req, context=ctx).read()
    soup = BeautifulSoup(html, "html.parser")
    
    iWant = getHTML(soup)
#     title = getChapterTitle(soup)
    
#     if title !=None:
    download(iWant, file_name)

In [51]:
## wanna be in a r/s


def getLink(Soup):
    try:
        link = soup.findAll('a',attrs={'id':'next_chap'})[0]['href']
    except:
        return False
    
    if link != None:
        return link
    return False

def getHTML(Soup): # clean up
    
    print()
    print("original iwant:\n")
    print(Soup)
    print()
    
    iWant = Soup.find('div', attrs={'id':'chapter-content'})
#     print(iWant)
#     Soup.findAll("div", {'class': "chapter-content"})
#     Soup.find('h1', {'class' : 'listing-name'})
#     print()
#     print("original iwant:\n")
#     print(Soup)
#     print()
    header = iWant.find('h2', attrs={'class':"c_title"})
#     print("header is: ", header.getText())
#     .find("h2", {'class': "c_title"})
#     iWant.find_all("h2", 'class'="c_title")

    removals = iWant.find('div', attrs={'class':'"ads ads-holder ads-middle text-center'})
#     find("div", {'class': "ads ads-holder ads-middle text-center"})
#     iWant.find_all('div', {'class':'ads ads-holder ads-middle text-center'})
    if removals != None:
        for match in removals:
            match.decompose()
        
        
    print()
    print("after ad cleaning iwant:\n")
    print(iWant)
    print()
    
    removals = iWant.find_all('div', {'align':'left'})
    if removals != None:
        for match in removals:
            match.decompose()
            
    print()
    print("removed alignment iwant:\n")
    print(iWant)
    print()
        
    text0 = header.getText()
#     print("after formatting:", text0[text0.index(":")+2:])
    header.replace_with('<h3>{}</h3>'.format(text0[text0.index(":")+2:]))
    print()
    print("completed iwant:\n")
    print(iWant)
    print()
    return  str(iWant) + "\n"


def createBook(file_name):
    file = open(file_name, "w", encoding="utf8")
    file.write('<?xml version="1.0" encoding="UTF-8" standalone="no"?>')
    file.write('\n<!DOCTYPE html>')
    file.write('\n<html xmlns:epub="http://www.idpf.org/2007/ops">')
    
    file.write("\n<head>")
    file.write('\n<meta charset="UTF-8"/>')
    file.write('\n<link href="epub_style_sheet.css" rel="stylesheet" type="text/css" />')
    file.close()

    
    

def next_chapter(link):
    req = Request(url=reg_url, headers=headers) 
    html = urlopen(req, context=ctx).read()
    soup = BeautifulSoup(html, "html.parser")
    return soup
    
def download(page, file_name, chapt_number):
    file = open(file_name, "a+", encoding="utf8")
    file.write("\n</head>")
    file.write("\n<body>")
    file.write("\n<p id = chapter{}>".format(str(chapt_number)))
    file.write(page)
    file.write("</p>")
    file.write("\n</body>")
#     file.write("\n</html>")
    file.close()

In [52]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import ssl

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}

reg_url = input('Enter - ')
soup =next_chapter(reg_url)
urlList =[]
chappieList =[]
ChaptNameList = []


file_name = "book3" + ".html"
createBook(file_name)
Count = 0

start_chapter = 6
end_chapter = 45  # loop through 45 times

while Count < 5:
    iWant = getHTML(soup)
    download(iWant, file_name, Count)
    
    
    link = getLink(soup)
    link = 'http://novelfull.com'+ link
#     print()
#     print("link is:",link)
    req = Request(url=link, headers=headers) 
    html = urlopen(req, context=ctx).read()
    soup = BeautifulSoup(html, "html.parser")
    Count +=1
    
file = open(file_name, "a+", encoding="utf8")
file.write("\n</html>")
file.close()
    
    
    
    
# https://novelfull.com/i-just-want-to-be-in-a-relationship/chapter-6.html

Enter - https://novelfull.com/i-just-want-to-be-in-a-relationship/chapter-6.html

original iwant:


<!DOCTYPE html>
<html lang="en-US"><head><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><meta content="IE=edge" http-equiv="X-UA-Compatible"/><meta content="width=device-width, height=device-height, initial-scale=1.0, user-scalable=0, minimum-scale=1.0, maximum-scale=1.0" name="viewport"/><meta content="_csrf" name="csrf-param"/><meta content="NjM2NGQzZjBmNDk1YjZhYjlkY2Y4ZDNiNWM2ZTBiMDEAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA==" name="csrf-token"/><!-- TODO: Meta --><meta content="Read I Just Want to Be in a Relationship - Chapter 6 online free - Novel Full" name="title"/><meta content="Read I Just Want to Be in a Relationship - Chapter 6 online free from your Mobile, Table, PC... Novel Updates Daily " name="description"/><meta content="https://novelfull.com/uploads/thumbs/i-just-want-to-be-in-a-relations-aa9f044739-30c864ee1a062dac423dc4e9bb552e97.jpg" name="imag


original iwant:


<!DOCTYPE html>
<html lang="en-US"><head><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><meta content="IE=edge" http-equiv="X-UA-Compatible"/><meta content="width=device-width, height=device-height, initial-scale=1.0, user-scalable=0, minimum-scale=1.0, maximum-scale=1.0" name="viewport"/><meta content="_csrf" name="csrf-param"/><meta content="NjM2NGQzZjBmNDk1YjZhYjlkY2Y4ZDNiNWM2ZTBiMDEAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA==" name="csrf-token"/><!-- TODO: Meta --><meta content="Read I Just Want to Be in a Relationship - Chapter 7 online free - Novel Full" name="title"/><meta content="Read I Just Want to Be in a Relationship - Chapter 7 online free from your Mobile, Table, PC... Novel Updates Daily " name="description"/><meta content="https://novelfull.com/uploads/thumbs/i-just-want-to-be-in-a-relations-aa9f044739-30c864ee1a062dac423dc4e9bb552e97.jpg" name="image"/><meta content="I Just Want to Be in a Relationship - Chapter 7" name="keyword


original iwant:


<!DOCTYPE html>
<html lang="en-US"><head><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><meta content="IE=edge" http-equiv="X-UA-Compatible"/><meta content="width=device-width, height=device-height, initial-scale=1.0, user-scalable=0, minimum-scale=1.0, maximum-scale=1.0" name="viewport"/><meta content="_csrf" name="csrf-param"/><meta content="NjM2NGQzZjBmNDk1YjZhYjlkY2Y4ZDNiNWM2ZTBiMDEAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA==" name="csrf-token"/><!-- TODO: Meta --><meta content="Read I Just Want to Be in a Relationship - Chapter 8 online free - Novel Full" name="title"/><meta content="Read I Just Want to Be in a Relationship - Chapter 8 online free from your Mobile, Table, PC... Novel Updates Daily " name="description"/><meta content="https://novelfull.com/uploads/thumbs/i-just-want-to-be-in-a-relations-aa9f044739-30c864ee1a062dac423dc4e9bb552e97.jpg" name="image"/><meta content="I Just Want to Be in a Relationship - Chapter 8" name="keyword


original iwant:


<!DOCTYPE html>
<html lang="en-US"><head><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><meta content="IE=edge" http-equiv="X-UA-Compatible"/><meta content="width=device-width, height=device-height, initial-scale=1.0, user-scalable=0, minimum-scale=1.0, maximum-scale=1.0" name="viewport"/><meta content="_csrf" name="csrf-param"/><meta content="NjM2NGQzZjBmNDk1YjZhYjlkY2Y4ZDNiNWM2ZTBiMDEAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA==" name="csrf-token"/><!-- TODO: Meta --><meta content="Read I Just Want to Be in a Relationship - Chapter 9 online free - Novel Full" name="title"/><meta content="Read I Just Want to Be in a Relationship - Chapter 9 online free from your Mobile, Table, PC... Novel Updates Daily " name="description"/><meta content="https://novelfull.com/uploads/thumbs/i-just-want-to-be-in-a-relations-aa9f044739-30c864ee1a062dac423dc4e9bb552e97.jpg" name="image"/><meta content="I Just Want to Be in a Relationship - Chapter 9" name="keyword


original iwant:


<!DOCTYPE html>
<html lang="en-US"><head><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><meta content="IE=edge" http-equiv="X-UA-Compatible"/><meta content="width=device-width, height=device-height, initial-scale=1.0, user-scalable=0, minimum-scale=1.0, maximum-scale=1.0" name="viewport"/><meta content="_csrf" name="csrf-param"/><meta content="NjM2NGQzZjBmNDk1YjZhYjlkY2Y4ZDNiNWM2ZTBiMDEAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA==" name="csrf-token"/><!-- TODO: Meta --><meta content="Read I Just Want to Be in a Relationship - Chapter 10 online free - Novel Full" name="title"/><meta content="Read I Just Want to Be in a Relationship - Chapter 10 online free from your Mobile, Table, PC... Novel Updates Daily " name="description"/><meta content="https://novelfull.com/uploads/thumbs/i-just-want-to-be-in-a-relations-aa9f044739-30c864ee1a062dac423dc4e9bb552e97.jpg" name="image"/><meta content="I Just Want to Be in a Relationship - Chapter 10" name="keyw